In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

In [2]:
from rembg import new_session, remove
from pathlib import Path
from tqdm.notebook import tqdm
from PIL import Image
from concurrent.futures import ThreadPoolExecutor
from itertools import repeat

In [3]:
session = new_session("u2net",  ['CUDAExecutionProvider'])
DATA_PATH = Path.cwd().parent.parent / "data"
REGENERATE = False

DATA_PATH

PosixPath('/workspaces/ds340-project/data')

In [4]:
session.inner_session.get_providers()

['CPUExecutionProvider']

In [5]:
import onnxruntime as ort
ort.get_available_providers()

['AzureExecutionProvider', 'CPUExecutionProvider']

In [6]:
def process_image(image_path: Path):
    sink = Path(str(image_path).replace('base_jpg', 'nobg_png').replace('jpg', 'png'))
    if REGENERATE or not sink.exists():
        with Image.open(image_path) as img:
            img = remove(img, session=session, bgcolor=(0, 0, 0, 0), post_process_mask=True)
            img.save(sink, "PNG")
    return

In [7]:
for dataset in ['food101-train', 'food101-validation']:
    all_files = list(Path(f'{DATA_PATH}/base_jpg/{dataset}').glob('*.jpg'))
    Path(f'{DATA_PATH}/base_jpg/{dataset}').mkdir(parents=True, exist_ok=True)
    Path(f'{DATA_PATH}/nobg_png/{dataset}').mkdir(parents=True, exist_ok=True)

    with tqdm(total=len(all_files), desc=dataset) as pbar:
        with ThreadPoolExecutor() as executor:
            for _ in executor.map(process_image, all_files):
                pbar.update(1)

    # for file in tqdm(all_files, desc=dataset):
    #     input_path = Path(file)
    #     output_path = Path(f'{DATA_PATH}/no_bg_jpg/{dataset}/{file.name}')

    #     if output_path.exists() and not REGENERATE: continue

    #     with open(input_path, 'rb') as i:
    #         image = Image.open(i)
    #         output = remove(image, session=session, bgcolor=(0, 0, 0, 0), post_process_mask=True)
    #         output.save(output_path, "PNG")

food101-train:   0%|          | 0/75750 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
Path(f'{DATA_PATH}/base_jpg/food101-train')

PosixPath('/workspaces/ds340-project/src/data/base_jpg/food101-train')

In [ ]:
all_files = list(Path(f'{DATA_PATH}/base_jpg/food101-train').glob('*.jpg'))
len(all_files)


0